In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
from utils.isomaxplus import IsoMaxPlusLossFirstPart, IsoMaxPlusLossSecondPart

from utils.misc import fix_random_seed
fix_random_seed(0)